In [17]:
def advanced_preprocessing_improved_fixed(
    train_df: pd.DataFrame, 
    test_df: pd.DataFrame, 
    target_col: str = 'итоговый_статус_займа',
    RANDOM_STATE: int = 42
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    """
    Улучшенная предобработка данных с сохранением всех параметров
    """
    print("="*60)
    print("НАЧАЛО ПРЕДОБРАБОТКИ (С СОХРАНЕНИЕМ ПАРАМЕТРОВ)")
    print("="*60)
    
    # Сохраняем параметры для воспроизводимости
    params = {
        'rating_order': {'А': 1, 'Б': 2, 'В': 3, 'Г': 4, 'Д': 5, 'Е': 6, 'Ж': 7},
        'experience_order': {
            '< 1 года': 0.5, '1 год': 1, '2 года': 2, '3 года': 3, 
            '4 года': 4, '5 лет': 5, '6 лет': 6, '7 лет': 7, 
            '8 лет': 8, '9 лет': 9, '10+ лет': 10
        },
        'binary_mapping': {'Да': 1, 'Нет': 0, 'Под вопросом': 0.5, 'True': 1, 'False': 0},
        'useless_cols': [
            'дата_следующей_выплаты',
            'кредитный_баланс_по_возоб_счетам',
            'совокупный_статус_подтверждения_доходов_заемщиков',
            'совокупный_пдн_заемщиков',
            'совокупный_доход_заемщиков',
        ],
        'constant_cols': ['платежный_график', 'особая_ситуация'],
        'binary_cols': ['пос_стоп_фактор', 'юридический_статус'],
        'onehot_cols': [
            'владение_жильем', 'подтвержден_ли_доход',
            'первоначальный_статус_займа', 'тип_займа',
            'тип_предоставления_кредита'
        ],
        'freq_cols': ['профессия_заемщика', 'допрейтинг', 'регион']
    }
    
    # Копируем данные
    train_df_processed = train_df.copy()
    test_df_processed = test_df.copy()
    
    # Сохраняем ID и target
    train_ids = train_df_processed['id'].copy()
    test_ids = test_df_processed['id'].copy()
    y = train_df_processed[target_col].copy() if target_col in train_df_processed.columns else None
    
    print(f"Начальные размеры: train={train_df_processed.shape}, test={test_df_processed.shape}")
    
    # 1. УДАЛЕНИЕ БЕСПОЛЕЗНЫХ ПРИЗНАКОВ
    print("\n1. Удаление бесполезных признаков...")
    cols_to_drop = [col for col in params['useless_cols'] + params['constant_cols'] 
                    if col in train_df_processed.columns]
    train_df_processed = train_df_processed.drop(columns=cols_to_drop, errors='ignore')
    test_df_processed = test_df_processed.drop(columns=cols_to_drop, errors='ignore')
    print(f"✓ Удалено {len(cols_to_drop)} признаков")
    
    # 2. ОБРАБОТКА ПРИЗНАКОВ
    print("\n2. Обработка признаков...")
    
    # 2.1 Обработка дат
    if 'дата_первого_займа' in train_df_processed.columns:
        for df in [train_df_processed, test_df_processed]:
            df['дата_первого_займа'] = pd.to_datetime(df['дата_первого_займа'], format='%m-%Y', errors='coerce')
            current_date = pd.Timestamp('2026-01-01')
            df['стаж_кредитной_истории_мес'] = ((current_date - df['дата_первого_займа']).dt.days / 30).fillna(0)
            df['стаж_кредитной_истории_мес'] = df['стаж_кредитной_истории_мес'].clip(0, 600)
            df.drop(columns=['дата_первого_займа'], inplace=True)
        print("✓ Создан признак: стаж_кредитной_истории_мес")
    
    # 2.2 Обработка сроков займа
    if 'срок_займа' in train_df_processed.columns:
        for df in [train_df_processed, test_df_processed]:
            df['срок_займа_мес'] = df['срок_займа'].str.extract(r'(\d+)').astype(float)
            df['срок_займа_мес'] = df['срок_займа_мес'] * 12
            df.drop(columns=['срок_займа'], inplace=True)
        print("✓ Создан признак: срок_займа_мес")
    
    # 2.3 Кодирование рейтингов
    if 'рейтинг' in train_df_processed.columns:
        for df in [train_df_processed, test_df_processed]:
            df['рейтинг_encoded'] = df['рейтинг'].map(params['rating_order']).fillna(0)
            df.drop(columns=['рейтинг'], inplace=True)
        print("✓ Закодирован рейтинг")
    
    # 2.4 Стаж работы (сохраняем медиану из train)
    if 'стаж' in train_df_processed.columns:
        # Кодируем train
        train_df_processed['стаж_encoded'] = train_df_processed['стаж'].map(params['experience_order'])
        median_experience = train_df_processed['стаж_encoded'].median()
        train_df_processed['стаж_encoded'] = train_df_processed['стаж_encoded'].fillna(median_experience)
        train_df_processed.drop(columns=['стаж'], inplace=True)
        
        # Кодируем test с той же медианой
        test_df_processed['стаж_encoded'] = test_df_processed['стаж'].map(params['experience_order'])
        test_df_processed['стаж_encoded'] = test_df_processed['стаж_encoded'].fillna(median_experience)
        test_df_processed.drop(columns=['стаж'], inplace=True)
        print("✓ Закодирован стаж")
    
    # 2.5 Бинарные признаки
    for col in params['binary_cols']:
        if col in train_df_processed.columns:
            for df in [train_df_processed, test_df_processed]:
                df[col] = df[col].map(params['binary_mapping']).fillna(0)
            print(f"✓ Закодирован {col}")
    
    # 2.6 One-Hot Encoding (сохраняем категории из train)
    onehot_categories = {}
    for col in params['onehot_cols']:
        if col in train_df_processed.columns:
            # Заполняем пропуски
            train_df_processed[col] = train_df_processed[col].fillna('MISSING')
            test_df_processed[col] = test_df_processed[col].fillna('MISSING')
            
            # Сохраняем уникальные категории из train
            unique_vals = train_df_processed[col].unique()
            onehot_categories[col] = list(unique_vals)
            
            # Создаем dummy-переменные для train
            dummies_train = pd.get_dummies(train_df_processed[col], prefix=col)
            
            # Создаем dummy-переменные для test
            dummies_test = pd.get_dummies(test_df_processed[col], prefix=col)
            
            # Выравниваем: добавляем отсутствующие в test
            for dummy_col in dummies_train.columns:
                if dummy_col not in dummies_test.columns:
                    dummies_test[dummy_col] = 0
            
            # Удаляем лишние из test
            for dummy_col in dummies_test.columns:
                if dummy_col not in dummies_train.columns:
                    dummies_test = dummies_test.drop(columns=[dummy_col])
            
            # Упорядочиваем как в train
            dummies_test = dummies_test[dummies_train.columns]
            
            # Объединяем
            train_df_processed = pd.concat([train_df_processed, dummies_train], axis=1)
            test_df_processed = pd.concat([test_df_processed, dummies_test], axis=1)
            
            # Удаляем исходный признак
            train_df_processed.drop(columns=[col], inplace=True)
            test_df_processed.drop(columns=[col], inplace=True)
            
            print(f"✓ One-hot: {col} ({len(dummies_train.columns)} категорий)")
    
    # 2.7 Frequency Encoding (сохраняем частоты из train)
    freq_encoders = {}
    for col in params['freq_cols']:
        if col in train_df_processed.columns:
            # Заполняем пропуски
            train_df_processed[col] = train_df_processed[col].fillna('MISSING')
            test_df_processed[col] = test_df_processed[col].fillna('MISSING')
            
            # Считаем частоты на train
            freq = train_df_processed[col].value_counts(normalize=True)
            freq_encoders[col] = freq
            
            # Применяем к train
            train_df_processed[f'{col}_freq_encoded'] = train_df_processed[col].map(freq)
            
            # Применяем к test
            test_df_processed[f'{col}_freq_encoded'] = test_df_processed[col].map(freq)
            
            # Заполняем пропуски в test средним значением частоты
            mean_freq = freq.mean() if len(freq) > 0 else 0
            test_df_processed[f'{col}_freq_encoded'] = test_df_processed[f'{col}_freq_encoded'].fillna(mean_freq)
            
            # Удаляем исходный признак
            train_df_processed.drop(columns=[col], inplace=True)
            test_df_processed.drop(columns=[col], inplace=True)
            
            print(f"✓ Frequency encoding: {col}")
    
    # 2.8 Цель займа
    if 'цель_займа' in train_df_processed.columns:
        purpose_groups = {
            'консолидация_долга': ['консолидация_долга'],
            'кредитная_карта': ['кредитная_карта'],
            'жилье': ['улучшение_жилищных_условий', 'дом'],
            'бизнес': ['мелкий_бизнес'],
            'авто': ['автомобиль'],
            'образование': ['образование'],
            'лечение': ['лечение'],
            'переезд': ['переезд'],
            'отпуск': ['отпуск'],
            'другое': ['другое', 'крупная_покупка', 'возобновляемая_энергия', 'свадьба']
        }
        
        purpose_to_group = {}
        for group, purposes in purpose_groups.items():
            for purpose in purposes:
                purpose_to_group[purpose] = group
        
        # Применяем группировку
        for df in [train_df_processed, test_df_processed]:
            df['цель_займа'] = df['цель_займа'].fillna('другое')
            df['цель_займа_группа'] = df['цель_займа'].map(purpose_to_group)
            df.loc[df['цель_займа_группа'].isna(), 'цель_займа_группа'] = 'другое'
        
        # Сохраняем уникальные группы из train
        unique_groups = train_df_processed['цель_займа_группа'].unique()
        
        # One-hot encoding
        dummies_train = pd.get_dummies(train_df_processed['цель_займа_группа'], prefix='цель_займа')
        dummies_test = pd.get_dummies(test_df_processed['цель_займа_группа'], prefix='цель_займа')
        
        # Выравниваем
        for dummy_col in dummies_train.columns:
            if dummy_col not in dummies_test.columns:
                dummies_test[dummy_col] = 0
        
        for dummy_col in dummies_test.columns:
            if dummy_col not in dummies_train.columns:
                dummies_test = dummies_test.drop(columns=[dummy_col])
        
        dummies_test = dummies_test[dummies_train.columns]
        
        # Объединяем
        train_df_processed = pd.concat([train_df_processed, dummies_train], axis=1)
        test_df_processed = pd.concat([test_df_processed, dummies_test], axis=1)
        
        # Удаляем исходные признаки
        train_df_processed.drop(columns=['цель_займа', 'цель_займа_группа'], inplace=True)
        test_df_processed.drop(columns=['цель_займа', 'цель_займа_группа'], inplace=True)
        
        print("✓ Обработана цель_займа")
    
    # 2.9 Обработка пени_за_дефолт (сохраняем медиану)
    if 'пени_за_дефолт' in train_df_processed.columns:
        for df in [train_df_processed, test_df_processed]:
            df['пени_за_дефолт'] = df['пени_за_дефолт'].map({'True': 1, 'False': 0})
        
        # Вычисляем медиану на train
        median_penalty = train_df_processed['пени_за_дефолт'].median()
        
        # Заполняем пропуски
        train_df_processed['пени_за_дефолт'] = train_df_processed['пени_за_дефолт'].fillna(median_penalty)
        test_df_processed['пени_за_дефолт'] = test_df_processed['пени_за_дефолт'].fillna(median_penalty)
        print("✓ Обработано пени_за_дефолт")
    
    # 3. СОЗДАНИЕ НОВЫХ ПРИЗНАКОВ
    print("\n3. Создание новых признаков...")
    
    # 3.1 Финансовые соотношения
    if all(col in train_df_processed.columns for col in ['аннуитет', 'годовой_доход']):
        for df in [train_df_processed, test_df_processed]:
            # Защита от деления на 0
            df['годовой_доход_safe'] = df['годовой_доход'].replace(0, 1)
            df['аннуитет_к_доходу'] = df['аннуитет'] * 12 / df['годовой_доход_safe']
            df.drop(columns=['годовой_доход_safe'], inplace=True)
        print("✓ Создан: аннуитет_к_доходу")
    
    if all(col in train_df_processed.columns for col in ['пдн', 'годовой_доход']):
        for df in [train_df_processed, test_df_processed]:
            df['годовой_доход_safe'] = df['годовой_доход'].replace(0, 1)
            df['пдн_от_дохода'] = df['пдн'] / df['годовой_доход_safe']
            df.drop(columns=['годовой_доход_safe'], inplace=True)
        print("✓ Создан: пдн_от_дохода")
    
    if all(col in train_df_processed.columns for col in ['сумма_займа', 'годовой_доход']):
        for df in [train_df_processed, test_df_processed]:
            df['годовой_доход_safe'] = df['годовой_доход'].replace(0, 1)
            df['заем_к_доходу'] = df['сумма_займа'] / df['годовой_доход_safe']
            df.drop(columns=['годовой_доход_safe'], inplace=True)
        print("✓ Создан: заем_к_доходу")
    
    # 4. ОБРАБОТКА ПРОПУСКОВ (сохраняем медианы)
    print("\n4. Обработка пропусков...")
    
    # Собираем медианы из train
    medians = {}
    numeric_cols_train = train_df_processed.select_dtypes(include=[np.number]).columns.tolist()
    
    for col in numeric_cols_train:
        if col in train_df_processed.columns and train_df_processed[col].isnull().any() and col != target_col:
            medians[col] = train_df_processed[col].median()
            train_df_processed[col] = train_df_processed[col].fillna(medians[col])
            
            # Заполняем test той же медианой
            if col in test_df_processed.columns:
                test_df_processed[col] = test_df_processed[col].fillna(medians[col])
    
    print(f"✓ Заполнены пропуски в {len(medians)} числовых признаках")
    
    # 5. УДАЛЕНИЕ ID И ВЫДЕЛЕНИЕ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ
    print("\n5. Подготовка финальных наборов данных...")
    
    # Выделяем целевую переменную из train
    if target_col in train_df_processed.columns:
        y = train_df_processed[target_col].copy()
        train_df_processed = train_df_processed.drop(columns=[target_col], errors='ignore')
    else:
        y = None
    
    # Удаляем ID
    X_train = train_df_processed.drop(columns=['id'], errors='ignore')
    X_test = test_df_processed.drop(columns=['id'], errors='ignore')
    
    # 6. УДАЛЕНИЕ КОНСТАНТНЫХ ПРИЗНАКОВ
    print("\n6. Удаление константных признаков...")
    constant_cols_final = [col for col in X_train.columns if X_train[col].nunique() == 1]
    if constant_cols_final:
        X_train = X_train.drop(columns=constant_cols_final, errors='ignore')
        X_test = X_test.drop(columns=[col for col in constant_cols_final if col in X_test.columns], errors='ignore')
        print(f"✓ Удалено {len(constant_cols_final)} константных признаков")
    
    # 7. ВЫРАВНИВАНИЕ КОЛОНОК
    print("\n7. Выравнивание колонок...")
    
    # Добавляем отсутствующие колонки в test
    missing_in_test = [col for col in X_train.columns if col not in X_test.columns]
    if missing_in_test:
        for col in missing_in_test:
            X_test[col] = 0
        print(f"✓ Добавлено {len(missing_in_test)} отсутствующих колонок в test")
    
    # Удаляем лишние колонки из test
    extra_in_test = [col for col in X_test.columns if col not in X_train.columns]
    if extra_in_test:
        X_test = X_test.drop(columns=extra_in_test, errors='ignore')
        print(f"✓ Удалено {len(extra_in_test)} лишних колонок из test")
    
    # Упорядочиваем колонки в test как в train
    X_test = X_test[X_train.columns]
    
    # 8. МАСШТАБИРОВАНИЕ (сохраняем scaler)
    print("\n8. Масштабирование признаков...")
    
    from sklearn.preprocessing import RobustScaler
    
    numeric_cols_to_scale = X_train.select_dtypes(include=[np.number]).columns.tolist()
    
    if numeric_cols_to_scale:
        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train[numeric_cols_to_scale])
        X_test_scaled = scaler.transform(X_test[numeric_cols_to_scale])
        
        X_train[numeric_cols_to_scale] = X_train_scaled
        X_test[numeric_cols_to_scale] = X_test_scaled
        
        print(f"✓ Масштабировано {len(numeric_cols_to_scale)} числовых признаков")
    else:
        print("⚠ Нет числовых признаков для масштабирования")
    
    print("\n" + "="*60)
    print("ПРЕДОБРАБОТКА ЗАВЕРШЕНА!")
    print("="*60)
    print(f"Итоговые размеры:")
    print(f"  X_train: {X_train.shape}")
    print(f"  X_test:  {X_test.shape}")
    print(f"  y_train: {y.shape if y is not None else 'None'}")
    
    if y is not None:
        print(f"  Баланс классов: 0={100*(y==0).mean():.1f}%, 1={100*(y==1).mean():.1f}%")
    
    return X_train, X_test, y, test_ids

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

RANDOM_STATE = 42

# Загружаем данные
train = pd.read_csv('../data/shift_ml_2026_train.csv', low_memory=False)
test = pd.read_csv('../data/shift_ml_2026_test.csv', low_memory=False)

print(f"Загружены данные: train={train.shape}, test={test.shape}")

# Применяем исправленную предобработку
X_train, X_test, y_train, test_ids = advanced_preprocessing_improved_fixed(
    train, test, target_col='итоговый_статус_займа', RANDOM_STATE=RANDOM_STATE
)

# Разделяем на train/val
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train
)

print(f"\nРазделение на train/val:")
print(f"  X_train_split: {X_train_split.shape}")
print(f"  X_val_split:   {X_val_split.shape}")
print(f"  y_train_split: {y_train_split.shape}")
print(f"  y_val_split:   {y_val_split.shape}")

# Обучаем модель
lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'n_estimators': 500,
    'learning_rate': 0.05,
    'max_depth': 7,
    'num_leaves': 31,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': -1
}

print("\nОбучение LightGBM...")
lgb_model = lgb.LGBMClassifier(**lgb_params)
lgb_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val_split, y_val_split)],
    eval_metric='auc',
    callbacks=[
        lgb.early_stopping(stopping_rounds=50, verbose=False),
        lgb.log_evaluation(period=100)
    ]
)

# Оценка на val
val_predictions = lgb_model.predict_proba(X_val_split)[:, 1]
val_auc = roc_auc_score(y_val_split, val_predictions)
print(f"\nVal AUC: {val_auc:.4f}")

# Переобучение на всех данных
print("\nПереобучение на всех train данных...")
final_model = lgb.LGBMClassifier(**lgb_params)
final_model.fit(X_train, y_train)
print(f"Финальная модель обучена на {len(X_train)} записях")

# Предсказания на тесте
test_predictions = final_model.predict_proba(X_test)[:, 1]

print(f"\nСтатистика предсказаний на тесте:")
print(f"  Min:  {test_predictions.min():.6f}")
print(f"  Max:  {test_predictions.max():.6f}")
print(f"  Mean: {test_predictions.mean():.6f}")
print(f"  Std:  {test_predictions.std():.6f}")

# Для сравнения - предсказания на train
train_predictions = final_model.predict_proba(X_train)[:, 1]
print(f"\nСтатистика предсказаний на train:")
print(f"  Min:  {train_predictions.min():.6f}")
print(f"  Max:  {train_predictions.max():.6f}")
print(f"  Mean: {train_predictions.mean():.6f}")
print(f"  Std:  {train_predictions.std():.6f}")

# Создаем сабмит
submission = pd.DataFrame({
    'ID': test_ids,
    'Proba': test_predictions
})

print("\n" + "="*60)
print("ФИНАЛЬНЫЙ САБМИТ")
print("="*60)
print(f"Размер: {submission.shape}")
print(f"Первые 10 строк:")
print(submission.head(10).to_string(index=False))

# Проверка на NaN
if submission['Proba'].isna().any():
    nan_count = submission['Proba'].isna().sum()
    print(f"\n⚠ ВНИМАНИЕ: Найдено {nan_count} NaN значений!")
    mean_val = submission['Proba'].mean()
    submission['Proba'] = submission['Proba'].fillna(mean_val)
    print(f"Заполнено средним значением: {mean_val:.6f}")

# Сохраняем
submission.to_csv('correct_final_submission.csv', index=False)
print(f"\n✅ Сабмит сохранен: correct_final_submission.csv")

Загружены данные: train=(1210779, 109), test=(134531, 108)
НАЧАЛО ПРЕДОБРАБОТКИ (С СОХРАНЕНИЕМ ПАРАМЕТРОВ)
Начальные размеры: train=(1210779, 109), test=(134531, 108)

1. Удаление бесполезных признаков...
✓ Удалено 7 признаков

2. Обработка признаков...
✓ Создан признак: стаж_кредитной_истории_мес
✓ Создан признак: срок_займа_мес
✓ Закодирован рейтинг
✓ Закодирован стаж
✓ Закодирован пос_стоп_фактор
✓ Закодирован юридический_статус
✓ One-hot: владение_жильем (6 категорий)
✓ One-hot: подтвержден_ли_доход (3 категорий)
✓ One-hot: первоначальный_статус_займа (2 категорий)
✓ One-hot: тип_займа (2 категорий)
✓ One-hot: тип_предоставления_кредита (2 категорий)
✓ Frequency encoding: профессия_заемщика
✓ Frequency encoding: допрейтинг
✓ Frequency encoding: регион
✓ Обработана цель_займа
✓ Обработано пени_за_дефолт

3. Создание новых признаков...


/home/yamshchikov/ML_practice/Torchvision_core_project/MLvenv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


✓ Создан: аннуитет_к_доходу
✓ Создан: пдн_от_дохода
✓ Создан: заем_к_доходу

4. Обработка пропусков...


/home/yamshchikov/ML_practice/Torchvision_core_project/MLvenv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


✓ Заполнены пропуски в 63 числовых признаках

5. Подготовка финальных наборов данных...

6. Удаление константных признаков...
✓ Удалено 3 константных признаков

7. Выравнивание колонок...

8. Масштабирование признаков...


/home/yamshchikov/ML_practice/Torchvision_core_project/MLvenv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1217: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a, func=_nanmedian, keepdims=keepdims,
/home/yamshchikov/ML_practice/Torchvision_core_project/MLvenv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1409: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(


✓ Масштабировано 94 числовых признаков

ПРЕДОБРАБОТКА ЗАВЕРШЕНА!
Итоговые размеры:
  X_train: (1210779, 119)
  X_test:  (134531, 119)
  y_train: (1210779,)
  Баланс классов: 0=80.0%, 1=20.0%

Разделение на train/val:
  X_train_split: (968623, 119)
  X_val_split:   (242156, 119)
  y_train_split: (968623,)
  y_val_split:   (242156,)

Обучение LightGBM...
[100]	valid_0's auc: 0.752749
[200]	valid_0's auc: 0.757696
[300]	valid_0's auc: 0.760124
[400]	valid_0's auc: 0.761504
[500]	valid_0's auc: 0.762164

Val AUC: 0.7622

Переобучение на всех train данных...
Финальная модель обучена на 1210779 записях

Статистика предсказаний на тесте:
  Min:  0.006307
  Max:  0.960932
  Mean: 0.198954
  Std:  0.156196

Статистика предсказаний на train:
  Min:  0.004281
  Max:  0.964006
  Mean: 0.199621
  Std:  0.156155

ФИНАЛЬНЫЙ САБМИТ
Размер: (134531, 2)
Первые 10 строк:
      ID    Proba
85540387 0.069873
28112500 0.046644
65731570 0.103155
65874747 0.532245
57893355 0.309357
80589347 0.351040
36381174 

In [21]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

# Кросс-валидация
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(
    lgb_model, X_train_split, y_train_split,
    cv=cv, scoring='roc_auc', n_jobs=-1
)

print(f"CV AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

CV AUC: 0.7599 (+/- 0.0006)
